# Tweet Cleanning and Classification

## Integrantes:

* Del Villar, Javier
* Otrino, Facundo Damián
* Pistoya, Haydeé Soledad
* Rojas, Mariano Arturo
* Sorza, Andrés
* Vaillard, Leandro Carlos

## Cargamos Librerías

In [1]:
import pandas as pd
import emoji
import re

## Cargamos dataset

El dataset contiene tweets de todos los idiomas y países sobre la pandemia COVID-19:

In [2]:
data = pd.read_csv('tweets_dataset.csv')

print(f'El dataset tiene {data.shape[0]} tweets!')

data.head()

El dataset tiene 155399 tweets!


,created_at,month,id,full_text,place_name
0,2020-01-30 21:01:21+00:00,2020-01,1222988192024088578,Hay alguna indicación sobre las precauciones q...,Ciudad Autónoma de Buenos Aires
1,2020-02-10 03:57:33+00:00,2020-02,1226716810839560192,en dos años ya nadie se va a acordar de estos ...,Ciudad Autónoma de Buenos Aires
2,2020-02-26 02:13:31+00:00,2020-02,1232488835855998976,Interesante artículo sobre el Coronavirus... h...,Choya
3,2020-02-26 22:39:04+00:00,2020-02,1232797255608029185,Lo que hubiera sido el sketch del cuñado de ma...,Corrientes
4,2020-02-06 13:38:16+00:00,2020-02,1225413401532403715,Quien le teme al Coronavirus ?? https://t.co/0...,El Calafate


## Preprocesamiento

In [3]:
def text_treatment(texto):
    """Esta función devuelve el texto tratado de las siguientes maneras:
    
    1) Sin Emojis
    
    2) Con Emojis transformados a texto
    
    3) Con Emojis transformados a texto entre comas
    """

    # Extractor de Emojis
    def emojis_extractor(text):
        a = ' '.join(c for c in text if c in emoji.UNICODE_EMOJI['es']).split()

        if a == []:
            return [text, text, text]
        else:
            text_SE = text
            text_CE = text
            text_CE_EC = text

            for i in a:
                #Emoji a texto
                emoji_to_replace = emoji.UNICODE_EMOJI['es'][i]

                # Elimina puntuacion en texto de emoji
                regex = '[\\:\\_]'
                emoji_to_replace = re.sub(regex , ' ', emoji_to_replace)

                # Texto Sin Emoji
                text_SE = text_SE.replace(i, ' ')[:512]
                
                # Con Emojis transformados a texto
                text_CE = text_CE.replace(i, ' ' + emoji_to_replace)[:512]
                
                # Con Emojis transformados a texto entre comas
                text_CE_EC = text_CE_EC.replace(i, ', ' + emoji_to_replace)[:512]
        
            treated_text = [text_SE, text_CE, text_CE_EC]
        
            return treated_text
    
    texto = emojis_extractor(texto)
        
    # Eliminación de espacios en blanco múltiples
    def space_remover(text):
        return re.sub('\\s+', ' ', text)

    texto = pd.Series(map(space_remover, texto))
    
    return texto

In [4]:
treated_text = data['full_text'].apply(text_treatment)
treated_text.columns = ['full_text_SE', 'full_text_CE', 'full_text_CE_EC']
treated_text

,full_text_SE,full_text_CE,full_text_CE_EC
0,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...
1,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...
2,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...
3,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...
4,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...
...,...,...,...
155394,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...
155395,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...
155396,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...
155397,"La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó..."


## Insertando las columnas de textp tratado en el dataset

In [5]:
data = pd.concat([data, treated_text], axis=1)
data

,created_at,month,id,full_text,place_name,full_text_SE,full_text_CE,full_text_CE_EC
0,2020-01-30 21:01:21+00:00,2020-01,1222988192024088578,Hay alguna indicación sobre las precauciones q...,Ciudad Autónoma de Buenos Aires,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...
1,2020-02-10 03:57:33+00:00,2020-02,1226716810839560192,en dos años ya nadie se va a acordar de estos ...,Ciudad Autónoma de Buenos Aires,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...
2,2020-02-26 02:13:31+00:00,2020-02,1232488835855998976,Interesante artículo sobre el Coronavirus... h...,Choya,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...
3,2020-02-26 22:39:04+00:00,2020-02,1232797255608029185,Lo que hubiera sido el sketch del cuñado de ma...,Corrientes,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...
4,2020-02-06 13:38:16+00:00,2020-02,1225413401532403715,Quien le teme al Coronavirus ?? https://t.co/0...,El Calafate,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...
...,...,...,...,...,...,...,...,...
155394,2021-04-11 01:25:20+00:00,2021-04,1381055733140221954,Pero nos encerraron todo el 2020 para “prepara...,San Isidro,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...
155395,2021-04-11 01:00:22+00:00,2021-04,1381049451205246979,A.C. (Antes del Covid) y D.C. (Después del Cov...,Lomas de Zamora,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...
155396,2021-04-10 23:58:48+00:00,2021-04,1381033955995095043,En un día sábado\n\nSe registraron 19.419 caso...,Colón,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...
155397,2021-04-11 01:32:40+00:00,2021-04,1381057580303273987,"La titular del PAMI, @luanavolnovich, comunicó...",Ituzaingó Centro,"La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó..."


## Clasificación de tweets

Inicialización del clasificador

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
  
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [7]:
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=0)

In [8]:
data['full_text_clasif'] = data['full_text'].apply(classifier)
data['full_text_SE_clasif'] = data['full_text_SE'].apply(classifier)
data['full_text_CE_clasif'] = data['full_text_CE'].apply(classifier)
data['full_text_CE_EC_clasif'] = data['full_text_CE_EC'].apply(classifier)

In [32]:
def score_extract(score):
    return int(score[0]['label'][0])

In [34]:
data['full_text_clasif'] = data['full_text_clasif'].apply(score_extract)
data['full_text_SE_clasif'] = data['full_text_SE_clasif'].apply(score_extract)
data['full_text_CE_clasif'] = data['full_text_CE_clasif'].apply(score_extract)
data['full_text_CE_EC_clasif'] = data['full_text_CE_EC_clasif'].apply(score_extract)
data

,created_at,month,id,full_text,place_name,full_text_SE,full_text_CE,full_text_CE_EC,full_text_clasif,full_text_SE_clasif,full_text_CE_clasif,full_text_CE_EC_clasif
0,2020-01-30 21:01:21+00:00,2020-01,1222988192024088578,Hay alguna indicación sobre las precauciones q...,Ciudad Autónoma de Buenos Aires,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...,Hay alguna indicación sobre las precauciones q...,3,3,3,3
1,2020-02-10 03:57:33+00:00,2020-02,1226716810839560192,en dos años ya nadie se va a acordar de estos ...,Ciudad Autónoma de Buenos Aires,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...,en dos años ya nadie se va a acordar de estos ...,1,1,1,1
2,2020-02-26 02:13:31+00:00,2020-02,1232488835855998976,Interesante artículo sobre el Coronavirus... h...,Choya,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...,Interesante artículo sobre el Coronavirus... h...,3,3,3,3
3,2020-02-26 22:39:04+00:00,2020-02,1232797255608029185,Lo que hubiera sido el sketch del cuñado de ma...,Corrientes,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...,Lo que hubiera sido el sketch del cuñado de ma...,1,1,1,1
4,2020-02-06 13:38:16+00:00,2020-02,1225413401532403715,Quien le teme al Coronavirus ?? https://t.co/0...,El Calafate,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...,Quien le teme al Coronavirus ?? https://t.co/0...,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
155394,2021-04-11 01:25:20+00:00,2021-04,1381055733140221954,Pero nos encerraron todo el 2020 para “prepara...,San Isidro,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...,Pero nos encerraron todo el 2020 para “prepara...,1,1,1,1
155395,2021-04-11 01:00:22+00:00,2021-04,1381049451205246979,A.C. (Antes del Covid) y D.C. (Después del Cov...,Lomas de Zamora,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...,A.C. (Antes del Covid) y D.C. (Después del Cov...,1,1,1,1
155396,2021-04-10 23:58:48+00:00,2021-04,1381033955995095043,En un día sábado\n\nSe registraron 19.419 caso...,Colón,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...,En un día sábado Se registraron 19.419 casos d...,1,1,1,1
155397,2021-04-11 01:32:40+00:00,2021-04,1381057580303273987,"La titular del PAMI, @luanavolnovich, comunicó...",Ituzaingó Centro,"La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó...","La titular del PAMI, @luanavolnovich, comunicó...",1,1,1,1


In [36]:
data.to_csv('classified_tweets.csv', index=False)